In [ ]:
import cupy as cp
import numpy as np
import zarr
import h5py
from decorrelation.shp import ks_test
from decorrelation.co import emperical_co, regularize_spectral, _isPD

## rslc

In [ ]:
rslc = cp.load('./rslc.npy')

In [ ]:
zarr.save('rslc.zarr', cp.asnumpy(rslc))

## coherence and covariance

In [ ]:
az_half_win = 5
r_half_win = 5
az_win = 2*az_half_win+1
r_win = 2*r_half_win+1

In [ ]:
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
del rmli
p = ks_test(sorted_rmli,az_half_win=az_half_win,r_half_win=r_half_win)[1]

In [ ]:
is_shp = (p < 0.05) & (p >= 0.0)
del p

In [ ]:
cov,coh = emperical_co(rslc,is_shp)
del rslc, is_shp

In [ ]:
coh.shape

(2500, 1834, 17, 17)

In [ ]:
%%time
zarr.save_array('emperical_coherence.zarr',cp.asnumpy(coh),chunks=(1000, 1000, 17, 17),compressor=None)

CPU times: user 5.29 s, sys: 18.6 s, total: 23.9 s
Wall time: 24.9 s


In [ ]:
%%time
# The result shows when setting propriate chunk size, the save speeds are almost save
# h5f = h5py.File('coherence.h5', 'w')
# h5f.create_dataset('coherence', data=cp.asnumpy(coh),chunks=(1000, 1000, 17, 17))
# h5f.close()

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 5.25 µs


In [ ]:
%%time
zarr.save_array('emperical_covariance.zarr',cp.asnumpy(cov),chunks=(1000, 1000, 17, 17),compressor=None)

CPU times: user 5.17 s, sys: 18.3 s, total: 23.4 s
Wall time: 24.4 s
